<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/Fundus-eye-disease-detection/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### test

In [ ]:
!pip install tensorflow
import numpy as np
!pip install opencv-python
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter
from google.colab import drive
from sklearn.model_selection import train_test_split
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define constants
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30  # Increased epochs for better training
CATEGORIES = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']
DATA_DIR = '/content/drive/MyDrive/data set/dataset'

# Data Augmentation & Data Generators with additional augmentations
train_val_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=30, width_shift_range=0.15, height_shift_range=0.15,
    shear_range=0.15, zoom_range=0.3, horizontal_flip=True, fill_mode='nearest',
    validation_split=0.25, brightness_range=[0.8, 1.2]
)

# Splitting dataset: 80% train, 16% validation, 4% test
train_generator = train_val_datagen.flow_from_directory(DATA_DIR, target_size=(IMG_SIZE, IMG_SIZE),
                                                        batch_size=BATCH_SIZE, class_mode='categorical', subset='training')

val_generator = train_val_datagen.flow_from_directory(DATA_DIR, target_size=(IMG_SIZE, IMG_SIZE),
                                                      batch_size=BATCH_SIZE, class_mode='categorical', subset='validation', shuffle=False)

# Handling Class Imbalance by computing class weights
class_counts = Counter(train_generator.classes)
total_samples = sum(class_counts.values())
class_weights = {i: total_samples / (len(CATEGORIES) * count) for i, count in class_counts.items()}

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)  # Increased units in the dense layer
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Keep Dropout at 0.5 for better generalization
predictions = Dense(len(CATEGORIES), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Callbacks
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * (0.1 ** (epoch // 10)), verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)  # Increased patience

# Train model
history = model.fit(train_generator, epochs=EPOCHS, validation_data=val_generator,
                    class_weight=class_weights, callbacks=[lr_schedule, early_stopping])

# Fine-tune last 12 layers (more layers for fine-tuning)
for layer in base_model.layers[-12:]:  # Fine-tuning more layers
    layer.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tuning
history_finetune = model.fit(train_generator, epochs=10, validation_data=val_generator,
                             class_weight=class_weights, callbacks=[lr_schedule, early_stopping])

# Evaluate on test set (validation set used here as a proxy)
test_loss, test_acc = model.evaluate(val_generator)  # Evaluate on validation set as a proxy for test set
print(f"Test Accuracy: {test_acc:.4f}")

# Save model
model.save('/content/vgg16_eye_disease_model_v2.h5')

# Confusion Matrix & Classification Report
y_pred = model.predict(val_generator)  # Use validation data for testing
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = val_generator.classes  # Use .classes instead of .labels
cm = confusion_matrix(y_true, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CATEGORIES, yticklabels=CATEGORIES)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

print(classification_report(y_true, y_pred_classes, target_names=CATEGORIES))


Mounted at /content/drive
Found 3173 images belonging to 4 classes.
Found 1055 images belonging to 4 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         131,3

 Total params: 14,848,068 (56.64 MB)

 Trainable params: 132,868 (519.02 KB)

 Non-trainable params: 14,715,200 (56.13 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: LearningRateScheduler setting learning rate to 0.0001.
Epoch 1/30
 18/100 ━━━━━━━━━━━━━━━━━━━━ 26:55 20s/step - accuracy: 0.2685 - loss: 1.7783

### New  predection

In [3]:
import tensorflow as tf
import cv2
import numpy as np



# Function to preprocess the image
def preprocess_image(img_path, img_size=224):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Could not read the image: {img_path}")
    img = cv2.resize(img, (img_size, img_size))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Load the saved model
model = tf.keras.models.load_model('/content/vgg16_eye_disease_model.h5')

# Path to the new image
image_path = ''
# Preprocess the image
try:
    img = preprocess_image(image_path)
except ValueError as e:
    print(f"Error: {e}")
    exit()

# Make the prediction
predictions = model.predict(img)
predicted_class_index = np.argmax(predictions)

# Get the predicted class label
CATEGORIES = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']
predicted_class_label = CATEGORIES[predicted_class_index]

# Print the prediction
print(f"Predicted class for {image_path}: {predicted_class_label}")

ModuleNotFoundError: No module named 'tensorflow'